In [2]:
import numpy as np
import radiomics
from radiomics import featureextractor



In [ ]:
#### GLCM

def extractor_setting(feature_names):
  """
  Settings of the feature extraction
  
  :param feature_names: list of the names of the features to be extracted
      e.g. features_names = ['JointEnergy', 'Contrast','Correlation']
  :return: extractor with pre-defined settings and the # of features (length of feature_names)
  """

  settings = {}
  settings['binWidth'] = 1 #To ensure a 1 pixel width for the histogram once the image has been requantized to 8 gray levels.
  settings['correctMask'] = True #To ensure the mask and the image are in the same coordinates
  # Instantiate the extractor
  extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
  extractor.disableAllFeatures()
  #extractor.enableFeatureClassByName('glcm') #Enable all GLCM features
  extractor.enableFeaturesByName(glcm=feature_names)
  num_features = len(feature_names)
  print('Extraction parameters:\n\t', extractor.settings)
  print('Enabled filters:\n\t', extractor.enabledImagetypes)
  print('Enabled features:\n\t', extractor.enabledFeatures)
  return extractor, num_features

def GLCM_feature_im(img_path, extractor, num_features):
  """
  Description: Extract GLCM features from an image. Requantization to 8 gray levels.

  :param img_path: Path of image from which the features willbe obtain
  :param extractor: Pyradiomics extractor defined in function extractor_settings
  :param num_features: # of features given by function extractor_settings.
  :return: feature vector
  """

  #Read image
  img = cv.cvtColor(cv.imread(img_path),cv.COLOR_BGR2GRAY)#Read and to grayscale
  im = np.uint8(np.digitize(img, np.arange(0,256, 32))) - 1 #Normalize to 8 gray level
  im_im = sitk.GetImageFromArray(im) #Pass to sitk objet
  #mask
  mask = np.ones(im.shape).astype(np.uint8) #Create maskwith same shape as image
  mask[-1,-1]=0 #Hack to allow full image segmentation. Last value is set to 0 so it wont be counted in the GLCM computation
  mask_im = sitk.GetImageFromArray(mask) #Pass to sitk object
  #Extract
  result = extractor.execute(im_im,mask_im) # Extract features
  features = np.array(list(result.values())[-num_features:]) #save as array
  return features

def GLCM_for_folder(paths, f, tt, mag, num_features, extractor):
  """
  Definition: Extract feature vector for all images in a folder, saved as a pickle numpy matrix

  :param paths: paths of images
  :param f: fold number-1
  :param tt: train or test encoded
  :param mag: magnification encoded using dictionary
  :param num_features: Number of features ot be extracted. It defines the size of the feature matrix
  :param extractor: Pyradiomics extractor with presettings
  :return: NONE (Pickle file will be saved in drive)
  """

  paths_folder = paths[f,tt,mag] #Paths of the first folder (There are 40)
  n = len(paths_folder) #number of images (samples)
  feature_matrix = np.zeros((n,num_features),dtype=np.float32) #Shape of feature matrix set

  #Extraction of features
  for i in tqdm(range(n)):
    img_path = paths_folder[i] #Image from which features are extracted
    ft_v = GLCM_feature_im(img_path, extractor, num_features) #Returns feature vector of m-dimension
    feature_matrix[i,:] = ft_v
  #Saving document
  file_name = f'/content/drive/MyDrive/Ars_machinae_autodiscentis/Inceptum/fold{f+1}/{tt_dict[tt]}/GLCM_f{f+1}_{tt_dict[tt]}_{mag_dict[mag]}x_fv.p'
  with open(file_name, 'wb') as handle:
      pickle.dump(feature_matrix, handle, protocol=pickle.HIGHEST_PROTOCOL)